In [1]:
import os

In [2]:
import cudf
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import gc
import subprocess

In [3]:
def get_ip():
    cmd = "hostname --all-ip-addresses"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    IPADDR = str(output.decode()).split()[0]
    print(IPADDR)
    return IPADDR

Unfortunately, what we tried to do here is not supported by UCX

In [4]:
def get_client(gpus):
    ip = get_ip()
    cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=gpus,
                               ip=ip)
    client = Client(cluster,name=f'GPU {gpus}')
    return client

In [5]:
gpu1 = get_client('1')
gpu2 = get_client('2')

100.96.201.45


2023-02-28 17:13:05,935 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-02-28 17:13:05,935 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


100.96.201.45


/opt/conda/envs/rapids/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41835 instead
  warnings.warn(
2023-02-28 17:13:09,230 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-02-28 17:13:09,230 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [6]:
gpu1

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://100.96.201.45:8787/status,
Dashboard: http://100.96.201.45:8787/status,Workers: 1
Total threads: 1,Total memory: 200.00 GiB
Status: running,Using processes: True
Comm: tcp://100.96.201.45:40557,Workers: 1
Dashboard: http://100.96.201.45:8787/status,Total threads: 1
Started: Just now,Total memory: 200.00 GiB
Comm: tcp://100.96.201.45:39813,Total threads: 1
Dashboard: http://100.96.201.45:45707/status,Memory: 200.00 GiB
Nanny: tcp://100.96.201.45:46429,


In [7]:
gpu2

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://100.96.201.45:41835/status,
Dashboard: http://100.96.201.45:41835/status,Workers: 1
Total threads: 1,Total memory: 200.00 GiB
Status: running,Using processes: True
Comm: tcp://100.96.201.45:46621,Workers: 1
Dashboard: http://100.96.201.45:41835/status,Total threads: 1
Started: Just now,Total memory: 200.00 GiB
Comm: tcp://100.96.201.45:46407,Total threads: 1
Dashboard: http://100.96.201.45:38531/status,Memory: 200.00 GiB
Nanny: tcp://100.96.201.45:39501,


In [8]:
PATH = '/raid/data/ml/kaggle/otto'

In [9]:
os.listdir(PATH)

['tr.parquet',
 'test.parquet',
 'te.parquet',
 'va_y.parquet',
 'train.parquet',
 'va_x.parquet']

In [10]:
%%time

train = gpu1.submit(cudf.read_parquet,f'{PATH}/train.parquet')
test = gpu2.submit(cudf.read_parquet,f'{PATH}/test.parquet')

CPU times: user 281 ms, sys: 118 ms, total: 399 ms
Wall time: 388 ms


In [11]:
train

<Future: pending, key: read_parquet-c88c2b8f04a7bc63fc8860a6ede5dd4e>

In [12]:
test

<Future: finished, type: cudf.core.dataframe.DataFrame, key: read_parquet-225d8a7f904d2e5b477df7bbb76eca4b>

In [13]:
train.client

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://100.96.201.45:8787/status,
Dashboard: http://100.96.201.45:8787/status,Workers: 1
Total threads: 1,Total memory: 200.00 GiB
Status: running,Using processes: True
Comm: tcp://100.96.201.45:40557,Workers: 1
Dashboard: http://100.96.201.45:8787/status,Total threads: 1
Started: Just now,Total memory: 200.00 GiB
Comm: tcp://100.96.201.45:39813,Total threads: 1
Dashboard: http://100.96.201.45:45707/status,Memory: 200.00 GiB
Nanny: tcp://100.96.201.45:46429,


It is tempting to run `train = train.result()` which will copy `train` from worker `gpu 1` to the manager `gpu 0`. It is not recommended unless necessary becuase if `train` is big, it would be pretty slow.

In [14]:
%%time

train_head = gpu1.submit(lambda df: df.head(), train)
train_head

CPU times: user 14.9 ms, sys: 0 ns, total: 14.9 ms
Wall time: 14.1 ms


<Future: pending, key: lambda-f495917eb19cb657058bd34d58305a31>

In [15]:
train_head.result()

,session,aid,ts,type
0,0,1517085,1659304800025,clicks
1,0,1563459,1659304904511,clicks
2,0,1309446,1659367439426,clicks
3,0,16246,1659367719997,clicks
4,0,1781822,1659367871344,clicks


In [16]:
# train.head()
gpu1.submit(lambda df: df.head(), train).result()

,session,aid,ts,type
0,0,1517085,1659304800025,clicks
1,0,1563459,1659304904511,clicks
2,0,1309446,1659367439426,clicks
3,0,16246,1659367719997,clicks
4,0,1781822,1659367871344,clicks


In [17]:
gpu2.submit(lambda df: df.head(), test).result()

,session,aid,ts,type
0,12899779,59625,1661724000278,clicks
1,12899780,1142000,1661724000378,clicks
2,12899780,582732,1661724058352,clicks
3,12899780,973453,1661724109199,clicks
4,12899780,736515,1661724136868,clicks


In [18]:
def encode_type(df):
    type_labels = {'clicks':0, 'carts':1, 'orders':2}
    df['type'] = df['type'].map(type_labels).astype('int8')
    return df

In [19]:
train = gpu1.submit(encode_type, train)
test = gpu2.submit(encode_type, test)

In [20]:
gpu1.submit(lambda df: df.head(), train).result()

,session,aid,ts,type
0,0,1517085,1659304800025,0
1,0,1563459,1659304904511,0
2,0,1309446,1659367439426,0
3,0,16246,1659367719997,0
4,0,1781822,1659367871344,0


In [21]:
def set_ts(df):
    df['ts'] = (df['ts']//1000).astype('int32')
    return df

In [22]:
train = gpu1.submit(set_ts, train)
test = gpu2.submit(set_ts, test)
train

<Future: pending, key: set_ts-8b106cbedc80996900f35c123c559cb2>

In contrast, we can't do this:
```
train['ts'] = gpu1.submit(lambda df: (df['ts']//1000).astype('int32'), train)
```

In [23]:
gpu1.submit(lambda df: df.head(), train).result()

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0


In [24]:
gpu2.submit(lambda df: df.head(), test).result()

,session,aid,ts,type
0,12899779,59625,1661724000,0
1,12899780,1142000,1661724000,0
2,12899780,582732,1661724058,0
3,12899780,973453,1661724109,0
4,12899780,736515,1661724136,0


In [25]:
%%time
gpu1.submit(lambda df: df.shape, train).result()

CPU times: user 16.6 ms, sys: 0 ns, total: 16.6 ms
Wall time: 17.5 ms


(216716096, 4)

The future `train` lives in `gpu 1` so it can't be accessed by `gpu 2`

In [26]:
gpu2.submit(lambda df: df.shape, test).result()

(6928123, 4)

In [27]:
%%time

gpu1.submit(lambda df: df.dtypes, train).result()

CPU times: user 13.2 ms, sys: 4.25 ms, total: 17.4 ms
Wall time: 23.4 ms


session    int32
aid        int32
ts         int32
type        int8
dtype: object

In [28]:
gpu1.submit(lambda df: df.memory_usage().sum(),train).result()/2**30

2.6238237023353577

In [29]:
gpu2.submit(lambda df: df.memory_usage().sum(),test).result()/2**30

0.08388012554496527

In [30]:
gpu1.futures

{'read_parquet-c88c2b8f04a7bc63fc8860a6ede5dd4e': <FutureState: finished>,
 'lambda-f495917eb19cb657058bd34d58305a31': <FutureState: finished>,
 'set_ts-8b106cbedc80996900f35c123c559cb2': <FutureState: finished>}

In [31]:
def remove_1_session(df):
    df['len'] = df.groupby('session')['aid'].transform('count')
    mask = df['len'] > 1
    ratio = mask.sum()/df.shape[0]
    print(f'keep {ratio:.3f} of df where len>1')
    df = df[mask]
    return df

def split(train,valid_ratio=0.13):
    train = train.sort_values(['session','ts']).reset_index(drop=True)
    sid = int(train['session'].max()*(1-valid_ratio))
    print('break at sid =', sid)
    
    mask = train['session'] >= sid
    va = train[mask]
    tr = train[~mask]
    
    m = tr['ts'] < va['ts'].min()
    ratio = m.sum()/tr.shape[0]
    print(f'keep {ratio:.3f} of tr to enforce time order')
    tr = tr[m]
    
    tr = remove_1_session(tr)
    va = remove_1_session(va)
    assert tr['ts'].max() < va['ts'].min()
    return tr,va

In [33]:
%%time
tr_va = gpu1.submit(split,train)
tr_va

CPU times: user 8.22 ms, sys: 43 µs, total: 8.26 ms
Wall time: 7.98 ms


<Future: pending, key: split-d3fd60241b55956c8ed2ac1de142bbc2>

In [33]:
def get_one_of_pair(tr_va,n):
    tr,va = tr_va
    if n == 0:
        return tr
    return va
va = gpu1.submit(get_one_of_pair, tr_va, 1)
va

<Future: pending, key: get_one_of_pair-c804c286fd642c6b505a16fe7aacceb0>

In [34]:
def split_x_y(va, fc=2):
    va['step'] = va.groupby('session').cumcount()
    va['len'] = va.groupby('session')['aid'].transform('count')
    assert va['len'].min() >= 2
    va['br'] = va['len'] - va['len']*fc//(fc+1)
    mask = va['step'] >= va['br']
    vay = va[mask].reset_index(drop=True)
    vax = va[~mask].reset_index(drop=True)
    
    vax.to_parquet(f'{PATH}/va_x.parquet')
    vay.to_parquet(f'{PATH}/va_y.parquet')

In [35]:
done = gpu1.submit(split_x_y, va)
done

<Future: pending, key: split_x_y-766f9efea29d5ae3b12b4e99ecf8409e>

In [36]:
done

<Future: finished, type: NoneType, key: split_x_y-766f9efea29d5ae3b12b4e99ecf8409e>

In [37]:
tr = gpu1.submit(get_one_of_pair, tr_va, 0)
tr

<Future: pending, key: get_one_of_pair-4c7194f1bf729badf774699fd0fe214c>

In [38]:
%%time
gpu1.submit(lambda df: df.to_parquet(f'{PATH}/tr.parquet'), tr).result()

CPU times: user 290 ms, sys: 40.8 ms, total: 330 ms
Wall time: 6.46 s


In [39]:
%%time
gpu2.submit(lambda df: df.to_parquet(f'{PATH}/te.parquet'), test).result()

CPU times: user 25.7 ms, sys: 1.48 ms, total: 27.2 ms
Wall time: 234 ms


In [40]:
gpu1.futures

{'read_parquet-c88c2b8f04a7bc63fc8860a6ede5dd4e': <FutureState: finished>,
 'lambda-f495917eb19cb657058bd34d58305a31': <FutureState: finished>,
 'set_ts-8b106cbedc80996900f35c123c559cb2': <FutureState: finished>,
 'split-5a82a9ce1d8a538ecb50f874605bbf71': <FutureState: finished>,
 'get_one_of_pair-c804c286fd642c6b505a16fe7aacceb0': <FutureState: finished>,
 'split_x_y-766f9efea29d5ae3b12b4e99ecf8409e': <FutureState: finished>,
 'get_one_of_pair-4c7194f1bf729badf774699fd0fe214c': <FutureState: finished>}

In [41]:
del train, tr_va, tr, va

In [42]:
gpu1.restart()

break at sid = 11222806
keep 0.825 of tr to enforce time order
keep 0.997 of df where len>1
keep 1.000 of df where len>1


2023-02-28 17:19:43,566 - distributed.nanny - WARNING - Restarting worker
2023-02-28 17:19:45,778 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-02-28 17:19:45,779 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://100.96.201.45:8787/status,
Dashboard: http://100.96.201.45:8787/status,Workers: 1
Total threads: 1,Total memory: 200.00 GiB
Status: running,Using processes: True
Comm: tcp://100.96.201.45:40557,Workers: 1
Dashboard: http://100.96.201.45:8787/status,Total threads: 1
Started: 6 minutes ago,Total memory: 200.00 GiB
Comm: tcp://100.96.201.45:39057,Total threads: 1
Dashboard: http://100.96.201.45:45799/status,Memory: 200.00 GiB
Nanny: tcp://100.96.201.45:46429,
